In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\Automated-Stock-Price-Prediction-Pipeline-A-Complete-MLOps-Lifecycle'

In [22]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
load_dotenv()
API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')

In [13]:
base_url = 'https://www.alphavantage.co/query'

params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': "IBM",
        'interval': "1min",
        'apikey': API_KEY,
        'outputsize': 'compact'
}

response = requests.get(base_url, params=params)

In [14]:
response.json()

{'Meta Data': {'1. Information': 'Intraday (1min) open, high, low, close prices and volume',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2024-11-21 19:59:00',
  '4. Interval': '1min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (1min)': {'2024-11-21 19:59:00': {'1. open': '222.2800',
   '2. high': '222.4000',
   '3. low': '222.2800',
   '4. close': '222.4000',
   '5. volume': '70'},
  '2024-11-21 19:58:00': {'1. open': '222.4000',
   '2. high': '222.4000',
   '3. low': '222.4000',
   '4. close': '222.4000',
   '5. volume': '20'},
  '2024-11-21 19:55:00': {'1. open': '221.8100',
   '2. high': '221.8100',
   '3. low': '221.8100',
   '4. close': '221.8100',
   '5. volume': '3'},
  '2024-11-21 19:49:00': {'1. open': '221.8101',
   '2. high': '221.8101',
   '3. low': '221.8101',
   '4. close': '221.8101',
   '5. volume': '4'},
  '2024-11-21 19:46:00': {'1. open': '221.8100',
   '2. high': '222.3852',
   '3. low': '221.8100',
   '4. close': '222.2600',
   

In [8]:
response.json()

{'Error Message': 'the parameter apikey is invalid or missing. Please claim your free API key on (https://www.alphavantage.co/support/#api-key). It should take less than 20 seconds.'}

In [15]:
def fetch_stock_data(symbol, interval='1min', outputsize='compact'):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': symbol,
        'interval': interval,
        'apikey': API_KEY,
        'outputsize': outputsize
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

In [16]:
def convert_to_dataframe(json_data):
    time_series = json_data.get(f'Time Series ({interval})', {})
    
    df = pd.DataFrame.from_dict(time_series, orient='index')
    
    df.columns = ['open', 'high', 'low', 'close', 'volume']
    
    df.index = pd.to_datetime(df.index)
    
    df = df.apply(pd.to_numeric)
    
    return df

Output size parameter chosed to be "full" to extract historical data that will be used to train the model. 

IBM symbol cuz we choosed here about IBM stock prices.

In [17]:
#Fetching 5-min interval stock data for IBM
symbol = 'IBM'
interval = '1min'  # Can be 1min, 5min, 15min, 30min, or 60min
output_size = 'full'  # 'compact' for last 100 points or 'full' for the entire history


json_data = fetch_stock_data(symbol, interval, output_size)

df = convert_to_dataframe(json_data)

In [18]:
df

,open,high,low,close,volume
2024-11-21 19:59:00,222.2800,222.4000,222.2800,222.4000,70
2024-11-21 19:58:00,222.4000,222.4000,222.4000,222.4000,20
2024-11-21 19:55:00,221.8100,221.8100,221.8100,221.8100,3
2024-11-21 19:49:00,221.8101,221.8101,221.8101,221.8101,4
2024-11-21 19:46:00,221.8100,222.3852,221.8100,222.2600,21
...,...,...,...,...,...
2024-10-23 04:04:00,230.0580,230.1770,230.0282,230.1770,28
2024-10-23 04:03:00,230.0877,230.0877,230.0877,230.0877,1
2024-10-23 04:02:00,230.0877,230.0877,230.0877,230.0877,2
2024-10-23 04:01:00,230.0381,230.0778,230.0183,230.0183,9


In [13]:
df.head()

,open,high,low,close,volume
2024-10-18 19:59:00,232.31,232.31,232.31,232.31,1
2024-10-18 19:56:00,232.31,232.31,232.31,232.31,5
2024-10-18 19:54:00,232.37,232.40,232.37,232.40,202
2024-10-18 19:50:00,232.30,232.30,232.30,232.30,5
2024-10-18 19:46:00,232.34,232.34,232.30,232.30,5


In [14]:
df.isna().sum()

open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [15]:
df.to_csv()

',open,high,low,close,volume\r\n2024-10-18 19:59:00,232.31,232.31,232.31,232.31,1\r\n2024-10-18 19:56:00,232.31,232.31,232.31,232.31,5\r\n2024-10-18 19:54:00,232.37,232.4,232.37,232.4,202\r\n2024-10-18 19:50:00,232.3,232.3,232.3,232.3,5\r\n2024-10-18 19:46:00,232.34,232.34,232.3,232.3,5\r\n2024-10-18 19:45:00,232.4,232.4,232.4,232.4,20\r\n2024-10-18 19:43:00,232.4,232.4,232.4,232.4,25\r\n2024-10-18 19:33:00,232.3987,232.3987,232.3987,232.3987,1\r\n2024-10-18 19:32:00,232.28,232.28,232.28,232.28,25\r\n2024-10-18 19:14:00,232.4,232.4,232.21,232.21,75\r\n2024-10-18 19:12:00,232.4,232.4,232.4,232.4,50\r\n2024-10-18 19:07:00,232.36,232.36,232.36,232.36,2\r\n2024-10-18 19:04:00,232.4,232.4,232.39,232.39,133\r\n2024-10-18 19:00:00,232.2,232.2,232.2,232.2,1456940\r\n2024-10-18 18:57:00,232.35,232.35,232.35,232.35,3\r\n2024-10-18 18:56:00,232.35,232.35,232.35,232.35,25\r\n2024-10-18 18:46:00,232.4497,232.4497,232.4497,232.4497,100\r\n2024-10-18 18:45:00,232.4,232.4,232.4,232.4,1\r\n2024-10-18 1

Data Ingestion Module

In [23]:
from pathlib import Path
from dataclasses import dataclass
import logging


In [17]:
%pwd

'c:\\Users\\Portable\\OneDrive\\Desktop\\MLOps School Project'

In [24]:
from src.StockPricePrediction.utils.common import read_yaml, create_directories
from src.StockPricePrediction.constants import *

In [25]:
@dataclass(frozen=True)
class DataIngestionConfig:
    symbol: str
    interval: str
    outputsize: str
    api_key: str
    base_url: str
    output_dir: Path

In [26]:
import requests
import pandas as pd
import logging
import os

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def fetch_stock_data(self):
        params = {
            'function': 'TIME_SERIES_INTRADAY',
            'symbol': self.config.symbol,
            'interval': self.config.interval,
            'apikey': self.config.api_key,
            'outputsize': self.config.outputsize
        }
        response = requests.get(self.config.base_url, params=params)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            logging.error(f"Error fetching data: {response.status_code}")
            return None

    def save_raw_data_to_csv(self, data):
        time_series_key = f'Time Series (1min)'  
        if time_series_key in data:
            df = pd.DataFrame.from_dict(data[time_series_key], orient='index')
            df.columns = ['open', 'high', 'low', 'close', 'volume']
            df.index = pd.to_datetime(df.index)
            df = df.apply(pd.to_numeric)

            # Save DataFrame to CSV
            csv_path = os.path.join(self.config.output_dir, f'{self.config.symbol}_stock_data.csv')
            df.to_csv(csv_path)
            logging.info(f"Raw data saved to {csv_path}")
            return csv_path
        else:
            logging.error("Time Series data not found in response")
            return None




In [27]:
class ConfigurationManager:
    def __init__(self, config_filepath: Path):
        self.config = read_yaml(config_filepath)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_collection']  # Accessing data_collection from config.yaml
        
        # Create DataIngestionConfig using values from config.yaml
        data_ingestion_config = DataIngestionConfig(
            symbol=self.config['data_collection']['stock_symbols'],  # Accessing stock_symbols directly
            interval=self.config['data_collection']['interval'],  # Default value, you can modify it in your yaml if needed
            outputsize=self.config['data_collection']['outputsize'],  # Default value, can also be configurable
            api_key=os.getenv('ALPHA_VANTAGE_API_KEY'),  # Assuming this is set in your .env file
            base_url=config['api_endpoint'],  # Use api_endpoint from the yaml
            output_dir=Path(self.config['data_paths']['raw_data']).parent  # Adjusted to save in the raw data directory
        )
        
        # Ensure the output directory exists
        create_directories([data_ingestion_config.output_dir])
        
        return data_ingestion_config

In [29]:
config_manager = ConfigurationManager(config_filepath=CONFIG_FILE_PATH)  # Update the path
data_ingestion_config = config_manager.get_data_ingestion_config()

data_ingestion = DataIngestion(config=data_ingestion_config)
stock_data = data_ingestion.fetch_stock_data()
stock_data

[2024-11-22 14:32:28,182: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-22 14:32:28,184: INFO: common: created directory at: data]


{'Meta Data': {'1. Information': 'Intraday (1min) open, high, low, close prices and volume',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2024-11-21 19:59:00',
  '4. Interval': '1min',
  '5. Output Size': 'Full size',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (1min)': {'2024-11-21 19:59:00': {'1. open': '222.2800',
   '2. high': '222.4000',
   '3. low': '222.2800',
   '4. close': '222.4000',
   '5. volume': '70'},
  '2024-11-21 19:58:00': {'1. open': '222.4000',
   '2. high': '222.4000',
   '3. low': '222.4000',
   '4. close': '222.4000',
   '5. volume': '20'},
  '2024-11-21 19:55:00': {'1. open': '221.8100',
   '2. high': '221.8100',
   '3. low': '221.8100',
   '4. close': '221.8100',
   '5. volume': '3'},
  '2024-11-21 19:49:00': {'1. open': '221.8101',
   '2. high': '221.8101',
   '3. low': '221.8101',
   '4. close': '221.8101',
   '5. volume': '4'},
  '2024-11-21 19:46:00': {'1. open': '221.8100',
   '2. high': '222.3852',
   '3. low': '221.8100',
   '4. close': '222.2600',
 

In [28]:
# Example of how to use the classes
try:
    config_manager = ConfigurationManager(config_filepath=CONFIG_FILE_PATH)  # Update the path
    data_ingestion_config = config_manager.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    stock_data = data_ingestion.fetch_stock_data()
    
    if stock_data:
        data_ingestion.save_raw_data_to_csv(stock_data)
except Exception as e:
    logging.error(f"An error occurred: {e}")

[2024-11-22 14:22:02,674: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-22 14:22:02,676: INFO: common: created directory at: data]
[2024-11-22 14:22:04,861: INFO: 406052314: Raw data saved to data\IBM_stock_data.csv]
